In [1]:
import pandas as pd
import numpy as np
import pickle

dataframes = pd.read_pickle('recom_data/user_category_dummy3.pkl')

df_counts = dataframes['view_counts']
df_users = dataframes['users']

with open('recom_data/category_dict.pkl', 'rb') as handle:
    category_dict = pickle.load(handle)

item_similarity = pd.read_pickle('recom_data/item_similarity.pkl')

In [2]:
view_matrix = df_counts.pivot(index='user', columns='category', values='count')

In [3]:
class MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        """
            K: 잠재요인(latent factor)의 수
            alpha: 학습률
            beta: 정규화 계수
            iterations: SGD의 계산을 할 때의 반복 횟수
            verbose: SGD의 중간 학습과정 출력 여부
        """
        self.R = np.array(ratings)
        
        item_id_index = []
        index_item_id = []
        for i, one_id in enumerate(ratings):
            item_id_index.append([one_id, i])
            index_item_id.append([i, one_id])
        self.item_id_index = dict(item_id_index)
        self.index_item_id = dict(index_item_id)
        user_id_index = []
        index_user_id = []
        for i, one_id in enumerate(ratings.T):
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id])
        self.user_id_index = dict(user_id_index)
        self.index_user_id = dict(index_user_id)
        
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose
        
    def rmse(self):
        """
            현재의 P행렬과 Q행렬을 가지고 Root Mean Squared Error(RMSE)를 계산
        """
        xs, ys = self.R.nonzero()
        # R에서 평점이 있는 (0이 아닌) 요소의 인덱스를 저장
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            # 평점이 존재하는 요소 (사용자 x, 아이템 y) 각각에 대해 아래 코드를 실행
            prediction = self.get_prediction(x, y)
            # 사용자 x, 아이템 y에 대한 평점 예측치를 계산
            self.predictions.append(prediction)
            # 예측값을 리스트에 추가
            self.errors.append(self.R[x, y] - prediction)
            # 실제값과 예측값의 차이를 계산해서 오차값 리스트에 추가
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        # errors를 사용해서 RMSE를 계산
        return np.sqrt(np.mean(self.errors**2))
    
    def train(self):
        """
            정해진 횟수만큼 반복하며 P, Q, bu, bd 값을 업데이트하는 함수
        """
        # Initializing user-feature and movie-feature matrix
        # P, Q 행렬을 임의의 값으로 채움
        # 평균이 0, 표준편차가 10/K인 정규분포를 갖는 난수로 초기화
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initializing the bias terms
        # 사용자 평가 경향(bu)를 0으로 초기화 / 크기는 사용자 수 (num_users)
        # 아이템 평가 경향(bd)를 0으로 초기화 / 크기는 아이템 수 (num_items)
        # 전체 평균 b를 구해서 저장
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])
        
        # List of training samples
        # 평점 행렬에서 값이 존재하는 요소의 인덱스들을 가져옴
        # SGD를 적용할 대상 (평점이 있는 요소의 인덱스와 평점)을 리스트로 저장
        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]
        
        # Stochastic gardient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            # samples를 임의로 섞음 => 어디서 시작하는지에 따라 수렴의 속도가 달라짐
            # 매 반복마다 다양한 시작점에서 시작하기!
            self.sgd()
            # sgd를 실행 / P, Q, bu, bd가 업데이트 됨
            rmse = self.rmse()
            training_process.append((i+1, rmse))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("반복: {0}; Train RMSE = {1:0.4f}".format(i+1, rmse))
        return training_process
    
    def get_prediction (self, i, j):
        """
            Rating prediction for user i and item j
            사용자 i의 아이템 j에 대한 평점을 예측하는 함수
        """
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
        
    def get_one_prediction(self, user_id, item_id):
        return self.get_prediction(self.user_id_index[user_id], self.item_id_index[item_id])
    
    def full_prediction(self):
        return self.b + self.b_u[:, np.newaxis] + self.b_d[np.newaxis, :] + self.P.dot(self.Q.T)

    def sgd(self):
        """
            Stochastic gardient descent to get optimized P and Q matrix
        """
        for i, j, r in self.samples:
            # samples에 있는 각 사용자-아이템-평점 세트에 대해서 SGD를 적용
            prediction = self.get_prediction(i, j)
            # 사용자 i, 아이템 j에 대한 평점 예측치를 구함
            e = (r - prediction)
            # 실제 평점 r과 비교해서 오차를 구함
            
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])
            # 사용자 평가 경향, 아이템 평가 경향 업데이트
            
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j, :])

In [4]:
view_matrix = df_counts.pivot(index='user', columns='category', values='count')
R_temp = view_matrix.fillna(0)
mf = MF(R_temp, K=100, alpha=0.01, beta=0.2, iterations=100, verbose=True)
# 인자 40개 / 반복 50회 / 학습률 0.001 / 정규화 계수 0.02로 모델을 학습합니다.
mf.train()
print(' ')

반복: 10; Train RMSE = 0.8660
반복: 20; Train RMSE = 0.8645
반복: 30; Train RMSE = 0.8632
반복: 40; Train RMSE = 0.8581
반복: 50; Train RMSE = 0.8479
반복: 60; Train RMSE = 0.8400
반복: 70; Train RMSE = 0.8357
반복: 80; Train RMSE = 0.8328
반복: 90; Train RMSE = 0.8309
반복: 100; Train RMSE = 0.8294
 


In [5]:
print(mf.full_prediction())
# 전체 user-category의 예측값들입니다.

[[1.90911285 1.92306463 2.03961867 ... 2.2366305  1.85770386 1.87148979]
 [2.26065369 2.28654936 2.43037866 ... 2.64670931 2.23876555 2.23480832]
 [1.91853277 1.9524703  2.05416874 ... 2.15235855 1.80949829 1.76922399]
 ...
 [1.44397159 1.50183204 1.51609749 ... 1.3678365  1.13431666 1.00397158]
 [1.44089223 1.49876127 1.51290293 ... 1.36432526 1.13100963 1.00058551]
 [1.44424659 1.50217184 1.51625641 ... 1.36740024 1.1342188  1.00366116]]


In [6]:
testers = [
    {'user_id': 90001,'gender': '남성','age': 17}, {'user_id': 90081,'gender': '남성','age': 22}, {'user_id': 90161,'gender': '남성','age': 32}, {'user_id': 90221,'gender': '남성','age': 42}, {'user_id': 90281,'gender': '남성','age': 52}, {'user_id': 90341,'gender': '남성','age': 62}, {'user_id': 90401,'gender': '남성','age': 72},
    {'user_id': 90501,'gender': '여성','age': 12}, {'user_id': 90521,'gender': '여성','age': 22}, {'user_id': 90541,'gender': '여성','age': 32}, {'user_id': 90561,'gender': '여성','age': 42}, {'user_id': 90581,'gender': '여성','age': 52}, {'user_id': 90601,'gender': '여성','age': 62}, {'user_id': 90621,'gender': '여성','age': 72}
]

# 각각의 그룹에서 한명씩 유저를 데려오고... 그 유저의 추천 카테고리를 출력해봤습니다.
# 그룹화가 아니라 개인화 추천입니다!!! 같은 그룹이어도 다른 추천 결과가 나옵니다.
for tester in testers:
    print(tester['gender'], tester['age'])
    temp_data = dict()
    
    # 전체 카테고리에 대해 예측값을 계산합니다.
    for category in R_temp:
        temp_data[category] = mf.get_one_prediction(tester['user_id'], category)

    # 예측값을 기준으로 정렬합니다.
    recomm_df = pd.DataFrame.from_dict([temp_data]).T.reset_index().rename(columns={'index': 'category', 0: 'pred'})
    temp_recomm_df = recomm_df.sort_values(by='pred', ascending=False)
    temp_recomm_df['category'] = temp_recomm_df['category'].map(lambda x: category_dict[x])
    print(temp_recomm_df)

남성 17
      category      pred
420      기타 문구  2.236631
7      남성 언더웨어  2.148835
408   문학/과학/경영  2.134182
6        남성 잠옷  2.133255
3      여성 언더웨어  2.120564
..         ...       ...
243        장난감  1.228271
245      세차 용품  1.215497
257         비데  1.212796
258         욕조  1.211935
248  기타 자동차 용품  1.191708

[423 rows x 2 columns]
남성 22
      category      pred
3      여성 언더웨어  2.445759
303   위생/건강 용품  2.441009
7      남성 언더웨어  2.412480
6        남성 잠옷  2.405142
315     유아동 의류  2.397305
..         ...       ...
245      세차 용품  1.544986
232      기타 공구  1.537590
258         욕조  1.527778
257         비데  1.524593
248  기타 자동차 용품  1.508205

[423 rows x 2 columns]
남성 32
      category      pred
315     유아동 의류  2.770781
303   위생/건강 용품  2.735368
325    유아동 주얼리  2.726737
3      여성 언더웨어  2.693417
145        PSP  2.683168
..         ...       ...
230         목공  1.844456
258         욕조  1.825528
257         비데  1.814941
232      기타 공구  1.813387
248  기타 자동차 용품  1.810875

[423 rows x 2 columns]
남성 42
    

In [7]:
# 아래는 테스트중인 코드입니다. (svd 알고리즘) / https://www.fun-coding.org/recommend_basic6.html

In [8]:
view_matrix = view_matrix.fillna(0)
matrix_values = view_matrix.values
user_view_mean = np.mean(matrix_values, axis=1)

matrix_user_mean = matrix_values - user_view_mean.reshape(-1, 1)

In [29]:
from scipy.sparse.linalg import svds

user, sigma, vt = svds(matrix_user_mean, k = 80)
sigma = np.diag(sigma)

svd_user_prediction = np.dot(np.dot(user, sigma), vt) + user_view_mean.reshape(-1, 1)


test = list(category_dict.items())
test.sort(key=lambda x:x[0])
# print(test)

test2 = [x[1] for x in test]
# print(test2)
results = []
for i in range(0, 1140, 20):
#     print(df_users.loc[i]['user'], df_users.loc[i]['gender'], df_users.loc[i]['age'], test2[svd_user_prediction[i].argmax()])
#     results.append([df_users.loc[i]['user'], df_users.loc[i]['gender'], df_users.loc[i]['age'], test2[svd_user_prediction[i].argmax()]])
#     print(df_users.loc[i]['user'], df_users.loc[i]['gender'], df_users.loc[i]['age'])
    result = svd_user_prediction[i]
    result_df = pd.DataFrame(result, columns=['pred'])
    result_df['category'] = result_df.index
    result_df['category'] = result_df['category'].apply(lambda x: test2[x])
    result_df = result_df.sort_values(by='pred', ascending=False)
#     print(result_df)
    results.append([df_users.loc[i]['user'], df_users.loc[i]['gender'], df_users.loc[i]['age'], result_df])
#     result.sort()
#     result = result[-10:]
#     print(result)
# print(len(svd_user_prediction[1139]))

results.sort(key=lambda x: x[0])
for result in results:
    print(result[0], result[1], result[2])
    print(result[3])

# results.sort(key=lambda x: x[0])
# for result in results:
#     print(result)

90001 M 10
         pred  category
193  4.233460  기타 거실 가구
201  4.128613     아동 책장
206  3.536523        책장
224  3.394818        액자
221  3.290532      블라인드
..        ...       ...
271 -0.381265    눈건강 용품
14  -0.388669     남성 구두
42  -0.394019        팔찌
44  -0.576211     여성 장갑
4   -0.627159     남성 상의

[423 rows x 2 columns]
90021 M 10
         pred  category
420  3.766659     기타 문구
409  3.060728    예술/디자인
408  2.989141  문학/과학/경영
404  2.966257    유아동 도서
419  2.216143     사무 용품
..        ...       ...
185 -0.275010     피부 케어
12  -0.327964    여성 실내화
114 -0.339688   휴대폰 케이스
44  -0.352518     여성 장갑
23  -0.370819       단지갑

[423 rows x 2 columns]
90041 M 10
         pred     category
380  3.158081       자전거 부품
386  3.123360           텐트
330  2.724182     기타 수영 용품
379  2.691279          자전거
401  2.658583        요가 용품
..        ...          ...
378 -0.307144  기타 스키/보드 용품
9   -0.322740        여성 구두
15  -0.329164      남성 캐주얼화
357 -0.416129          탁구대
384 -0.433575       낚시 소모품

[423 rows x 2 colu

         pred   category
198  3.685600        수납장
188  3.554118         장롱
204  3.546383   기타 아동 가구
223  3.514734         시계
213  3.235746  기타 DIY 자재
..        ...        ...
125 -0.314522   모니터 주변기기
123 -0.379003    PC 액세서리
41  -0.397756        팬던트
0   -0.476693      여성 상의
16  -0.656528     남성 운동화

[423 rows x 2 columns]
90741 F 40
         pred   category
200  4.117832   아동 책상/의자
193  3.694100   기타 거실 가구
227  3.581844         솜류
211  3.475366         벽지
228  3.355975         수예
..        ...        ...
29  -0.250710  기타 헤어액세서리
5   -0.259763      남성 하의
419 -0.307009      사무 용품
116 -0.332808    휴대폰 충전기
144 -0.376616    플레이스테이션

[423 rows x 2 columns]
90761 F 40
         pred   category
1    4.285182      여성 하의
32   3.975634      공용 모자
0    3.826002      여성 상의
38   3.725783         반지
43   3.640756    기타 액세서리
..        ...        ...
131 -0.288743        마우스
45  -0.334988      남성 장갑
23  -0.419728        단지갑
207 -0.453385  기타 사무용 가구
14  -0.688804      남성 구두

[423 rows x 2 columns]
90781 